# Customer Churn Analysis
### Team-5
#### Aishwarya Paruchuri
#### Archita Chakraborty
#### Manjushree Barike Rajanna
#### Rohit Chandra

 ** Python statements that import the standard libraries **

In [1]:
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
#read the csv file in a dataframe
churn_df = pd.read_csv("../data/Customer_churn_raw.csv")

churn_df.head()

Call  Failure Complains  Subscription  Length Charge  Amount Seconds of Use  \
0             8         0                  38.0              0           4370   
1             O         0                  39.0              0            NaN   
2            10         0                  37.0              0           2453   
3            10         0                  38.0              0           4198   
4             3         0                   NaN              0           2393   

   Frequency of use  Frequency of SMS  Distinct Called Numbers  Tariff Plan  \
0              71.0               5.0                     17.0            1   
1               5.0               7.0                      4.0            1   
2              60.0             359.0                     24.0            1   
3              66.0               1.0                     35.0            1   
4              58.0               2.0                     33.0            1   

   Status  Age  Customer Value         FN        FP  Churn  
0       1    5         197.640   177.8760   69.7640      0  
1       1    6          46.035    41.4315   60.0000      0  
2       1    2        1536.520  1382.8680  203.6520      0  
3       1   10         240.020   216.0180   74.0020      0  
4       1    9         145.805   131.2245   64.5805      0

In [3]:
#rename the columns
churn_df=churn_df.rename(columns={"Call  Failure": "call_failure", "Complains": "complains", "Subscription  Length": "subs_len", "Charge  Amount": "charge_amount",
                   "Seconds of Use": "seconds_of_use", "Frequency of use": "freq_of_use", "Frequency of SMS": "freq_of_sms", "Distinct Called Numbers": "distinct_call_nums",
                   "Age Group": "age_group", "Tariff Plan": "tariff_plan", "Status": "status", "Age": "age", "Customer Value": "customer_value"})

In [4]:
#print the columns
churn_df.columns

Index(['call_failure', 'complains', 'subs_len', 'charge_amount',
       'seconds_of_use', 'freq_of_use', 'freq_of_sms', 'distinct_call_nums',
       'tariff_plan', 'status', 'age', 'customer_value', 'FN', 'FP', 'Churn'],
      dtype='object')

## Exploratory Data Analysis

In [5]:
#filter out the 3 columns from the dataframe
cols_3 = churn_df[['call_failure', 'complains', 'subs_len']]

cols_3

call_failure complains  subs_len
0               8         0      38.0
1               O         0      39.0
2              10         0      37.0
3              10         0      38.0
4               3         0       NaN
...           ...       ...       ...
3145           21         0      19.0
3146           17         0      17.0
3147           13         0      18.0
3148            7         0      11.0
3149            8         1      11.0

[3150 rows x 3 columns]

In [6]:
cols_3.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3150 entries, 0 to 3149
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   call_failure  3138 non-null   object 
 1   complains     3137 non-null   object 
 2   subs_len      3145 non-null   float64
dtypes: float64(1), object(2)
memory usage: 74.0+ KB


In [7]:
cols_3.describe()

subs_len
count  3145.000000
mean     32.333229
std       9.346742
min     -40.000000
25%      29.000000
50%      35.000000
75%      38.000000
max      47.000000

### Check for null values

In [8]:
#check for null values in 
cols_3.isna().sum()

call_failure    12
complains       13
subs_len         5
dtype: int64

Observation : 

12 null values in call_failure,

13 null values in complains,

5 null values in subs_len

### Data cleaning

In [9]:
cols_3['call_failure'].value_counts()

0             664
5             244
7             166
6             161
8             156
9             149
3             140
2             137
4             133
11            125
10            124
1             121
13             99
12             97
14             85
15             69
16             67
18             53
17             45
20             37
19             33
21             29
23             25
25             21
22             21
28             17
26             17
30             16
24             16
27             13
O              13
32              8
29              7
31              6
o               4
-0.0000009      4
34              3
33              3
35              2
36              2
3$^             1
0.05            1
0.000001        1
O'              1
ooo             1
-0.05           1
Name: call_failure, dtype: int64

**Observation:** Since the data is not consistent, we'll replace these values

In [10]:
cols_3['call_failure'] = pd.to_numeric(cols_3['call_failure'].astype(str).str.replace(',',''), errors='coerce').fillna(0).astype(int)

In [11]:
#replace null values with 0
cols_3["call_failure"].replace(np.nan, 0, inplace=True )

In [12]:
cols_3['complains'].value_counts()

0            2859
1             231
OO             18
o              12
O               7
-1              4
11              1
-112            1
-11             1
1999            1
18-             1
111111111       1
Name: complains, dtype: int64

 **Observation:** Since complains is boolean type we will chave to replace with weither 0 or 1

In [13]:
cols_3['complains'] = pd.to_numeric(cols_3['complains'].astype(str), errors='coerce').fillna(0).astype(int)

In [14]:
cols_3['complains'].value_counts()

 0            2910
 1             231
-1               4
 11              1
-112             1
-11              1
 1999            1
 111111111       1
Name: complains, dtype: int64

In [15]:
cols_3.loc[cols_3['complains'] != 0, 'complains'] = 1

In [16]:
# Verification
cols_3['complains'].value_counts()

0    2910
1     240
Name: complains, dtype: int64

In [17]:
#replace all the null values with 1
cols_3["call_failure"].replace(np.nan, 1, inplace=True )

In [18]:
#Clean the subscription length column
cols_3['subs_len'].value_counts()

 36.0000    272
 38.0000    257
 37.0000    226
 35.0000    225
 34.0000    201
 39.0000    198
 40.0000    182
 33.0000    152
 32.0000    121
 41.0000    110
 31.0000    104
 30.0000     80
 42.0000     80
 29.0000     73
 28.0000     59
 27.0000     56
 43.0000     56
 25.0000     52
 26.0000     45
 44.0000     44
 24.0000     40
 23.0000     39
 17.0000     32
 15.0000     31
 18.0000     30
 16.0000     30
 19.0000     29
 20.0000     28
 22.0000     26
 11.0000     26
 21.0000     25
 14.0000     25
 13.0000     24
 45.0000     23
 9.0000      22
 7.0000      19
 12.0000     19
 10.0000     16
 46.0000     13
 8.0000      12
 3.0000       8
 6.0000       8
 5.0000       6
 4.0000       4
 36.0009      4
-35.0000      3
-37.0000      3
-40.0000      3
 39.0010      3
 47.0000      1
Name: subs_len, dtype: int64

In [19]:
cols_3['subs_len'] = cols_3['subs_len'].abs()

In [20]:
cols_3['subs_len'].value_counts()

36.0000    272
38.0000    257
37.0000    229
35.0000    228
34.0000    201
39.0000    198
40.0000    185
33.0000    152
32.0000    121
41.0000    110
31.0000    104
30.0000     80
42.0000     80
29.0000     73
28.0000     59
43.0000     56
27.0000     56
25.0000     52
26.0000     45
44.0000     44
24.0000     40
23.0000     39
17.0000     32
15.0000     31
16.0000     30
18.0000     30
19.0000     29
20.0000     28
22.0000     26
11.0000     26
14.0000     25
21.0000     25
13.0000     24
45.0000     23
9.0000      22
12.0000     19
7.0000      19
10.0000     16
46.0000     13
8.0000      12
3.0000       8
6.0000       8
5.0000       6
36.0009      4
4.0000       4
39.0010      3
47.0000      1
Name: subs_len, dtype: int64

In [21]:
cols_3['subs_len'].fillna(value=cols_3['subs_len'].mean(), inplace=True)

In [22]:
cols_3['subs_len'] = cols_3['subs_len'].astype(int)

In [23]:
cols_3.dtypes

call_failure    int64
complains       int64
subs_len        int64
dtype: object

In [24]:
cols_3['subs_len'].value_counts()

36    276
38    257
37    229
35    228
34    201
39    201
40    185
33    152
32    126
41    110
31    104
30     80
42     80
29     73
28     59
27     56
43     56
25     52
26     45
44     44
24     40
23     39
17     32
15     31
18     30
16     30
19     29
20     28
11     26
22     26
21     25
14     25
13     24
45     23
9      22
7      19
12     19
10     16
46     13
8      12
3       8
6       8
5       6
4       4
47      1
Name: subs_len, dtype: int64

In [25]:
# Verification for null values

cols_3.isna().sum()

call_failure    0
complains       0
subs_len        0
dtype: int64

In [26]:
# Merging these clean columns to the original dataframe

In [27]:
churn_df = churn_df.drop(['call_failure', 'complains', 'subs_len'],axis = 1)
churn_df = pd.concat([cols_3,churn_df],axis = 1)

In [28]:
churn_df.isnull().sum()

call_failure           0
complains              0
subs_len               0
charge_amount          2
seconds_of_use        39
freq_of_use           19
freq_of_sms            4
distinct_call_nums     2
tariff_plan            0
status                 0
age                    0
customer_value        92
FN                     0
FP                     0
Churn                  0
dtype: int64

### Manjushree 

In [29]:
#Filter the dataframe for cleaning and analysis

col_4_5_12 = churn_df.filter(['charge_amount','customer_value','seconds_of_use','Churn'], axis=1)


In [30]:
#Summary of the dataframe

col_4_5_12.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3150 entries, 0 to 3149
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   charge_amount   3148 non-null   object 
 1   customer_value  3058 non-null   float64
 2   seconds_of_use  3111 non-null   object 
 3   Churn           3150 non-null   int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 98.6+ KB


In [31]:
#count the total number of null values in all the columns

col_4_5_12.isnull().sum()

charge_amount      2
customer_value    92
seconds_of_use    39
Churn              0
dtype: int64

### Observations:

Number of null values 

charge_amount : 2

customer_value: 92

Seconds_of_use : 39

Churn:   0

In [32]:
# Count the unique values in 'customer_value' column

col_4_5_12['customer_value'].value_counts()

0.000      40
45.495     11
40.440     10
15.165      6
25.275      5
           ..
99.520      1
119.565     1
165.520     1
167.700     1
868.140     1
Name: customer_value, Length: 2654, dtype: int64

In [33]:
# Replace the null values with zero

col_4_5_12['customer_value'].replace(np.nan, 0, inplace=True )

In [34]:
# Replace the null values with zero

col_4_5_12['charge_amount'].replace(np.nan, 0, inplace=True )

In [35]:
col_4_5_12['charge_amount'].value_counts()

0          1755
1           616
2           393
3           198
4            76
5            30
8            19
7            14
9            13
6            11
10            7
0             2
o2            1
o8            1
o7            1
o6            1
o5            1
o4            1
o3            1
-9            1
o1            1
o0            1
2.00001       1
o             1
-1            1
OOO           1
-3            1
OO            1
Name: charge_amount, dtype: int64

#### Observations:

Charge_amount has String values and negative values which needs to be replaced

In [36]:
#Replace incorrect values with correct values

col_4_5_12['charge_amount'].replace(['OOO','OO','0','o','o0'], 0, inplace=True )
col_4_5_12['charge_amount'].replace(['o1'], 1, inplace=True )
col_4_5_12['charge_amount'].replace(['o7'], 7, inplace=True )
col_4_5_12['charge_amount'].replace(['o2'], 2, inplace=True )
col_4_5_12['charge_amount'].replace(['o4'], 4, inplace=True )
col_4_5_12['charge_amount'].replace(['o8'], 8, inplace=True )
col_4_5_12['charge_amount'].replace(['o5'], 5, inplace=True )
col_4_5_12['charge_amount'].replace(['o3'], 3, inplace=True )
col_4_5_12['charge_amount'].replace(['o6'], 6, inplace=True )
col_4_5_12['charge_amount'].replace(['2.00001'], 2, inplace=True )

In [37]:
# Convert datatype to intgere and make negatives positive

col_4_5_12['charge_amount'] = col_4_5_12['charge_amount'].astype(int)
col_4_5_12['charge_amount'] = col_4_5_12['charge_amount'].abs()

In [38]:
# Count the unique values in 'charge_amount' column

col_4_5_12['charge_amount'].value_counts()

0     1761
1      618
2      395
3      200
4       77
5       31
8       20
7       15
9       14
6       12
10       7
Name: charge_amount, dtype: int64

In [39]:
# Count the unique values in 'seconds_of_use' column

col_4_5_12['seconds_of_use'].value_counts()

0       131
305      37
710       9
1015      9
2088      9
       ... 
6028      1
1330      1
860       1
335       1
1082      1
Name: seconds_of_use, Length: 1763, dtype: int64

In [40]:
#Replace the null values with zeros

col_4_5_12['seconds_of_use'].replace(np.nan, 0, inplace=True )

In [41]:
# Convert datatype to intgere and make negatives positive

col_4_5_12['seconds_of_use'].replace(['O3915'], 3915, inplace=True )
col_4_5_12['seconds_of_use'] = col_4_5_12['seconds_of_use'].astype(int)
col_4_5_12['seconds_of_use'] = col_4_5_12['seconds_of_use'].abs()

In [42]:
# Count the unique values in 'seconds_of_use' column

col_4_5_12['seconds_of_use'].value_counts()

0       170
305      37
1973      9
1015      9
710       9
       ... 
3973      1
2228      1
4025      1
2488      1
1082      1
Name: seconds_of_use, Length: 1750, dtype: int64

In [43]:
# Count the total number fo null values 

col_4_5_12.isnull().sum()

charge_amount     0
customer_value    0
seconds_of_use    0
Churn             0
dtype: int64

In [44]:
# Get the summary of dataframe

col_4_5_12.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3150 entries, 0 to 3149
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   charge_amount   3150 non-null   int64  
 1   customer_value  3150 non-null   float64
 2   seconds_of_use  3150 non-null   int64  
 3   Churn           3150 non-null   int64  
dtypes: float64(1), int64(3)
memory usage: 98.6 KB


## AISHWARYA

In [45]:
col_4_5_12 = col_4_5_12.drop(['Churn'],axis = 1)
churn_df = churn_df.drop(['charge_amount','seconds_of_use'],axis = 1)
churn_df = churn_df.drop(['customer_value'],axis = 1)
churn_df = pd.concat([col_4_5_12,churn_df],axis = 1)

In [46]:
churn_df = churn_df[['call_failure', 'complains', 'subs_len', 'charge_amount',
       'seconds_of_use', 'freq_of_use', 'freq_of_sms', 'distinct_call_nums',
       'tariff_plan', 'status', 'age', 'customer_value', 'FN', 'FP', 'Churn']]
churn_df.head()

call_failure  complains  subs_len  charge_amount  seconds_of_use  \
0             8          0        38              0            4370   
1             0          0        39              0               0   
2            10          0        37              0            2453   
3            10          0        38              0            4198   
4             3          0        32              0            2393   

   freq_of_use  freq_of_sms  distinct_call_nums  tariff_plan  status  age  \
0         71.0          5.0                17.0            1       1    5   
1          5.0          7.0                 4.0            1       1    6   
2         60.0        359.0                24.0            1       1    2   
3         66.0          1.0                35.0            1       1   10   
4         58.0          2.0                33.0            1       1    9   

   customer_value         FN        FP  Churn  
0         197.640   177.8760   69.7640      0  
1          46.035    41.4315   60.0000      0  
2        1536.520  1382.8680  203.6520      0  
3         240.020   216.0180   74.0020      0  
4         145.805   131.2245   64.5805      0

In [47]:
# Check for null values
churn_df.isnull().sum()

call_failure           0
complains              0
subs_len               0
charge_amount          0
seconds_of_use         0
freq_of_use           19
freq_of_sms            4
distinct_call_nums     2
tariff_plan            0
status                 0
age                    0
customer_value         0
FN                     0
FP                     0
Churn                  0
dtype: int64

### Check for null values in freq_of_use , freq_of_sms columns and distinct_call_nums

Description about the columns:
1. Frequency of use: total number of calls
2. Frequency of SMS: total number of text messages
3. Distinct Called Numbers: total number of distinct phone calls

In [48]:
# Checking for null values in freq_of_use column
print('Number of null values in freq_of_use:',churn_df['freq_of_use'].isnull().sum())
# Checking for null values in freq_of_sms column
print('Number of null values in freq_of_sms:',churn_df['freq_of_sms'].isnull().sum())
# Checking for null values in distinct_call_nums column
print('Number of null values in distinct_call_nums:',churn_df['distinct_call_nums'].isnull().sum())

Number of null values in freq_of_use: 19
Number of null values in freq_of_sms: 4
Number of null values in distinct_call_nums: 2


In [49]:
# Checking if there are any negative values or extreme values. As these columns cant have negative numbers as values
print('Maximum value in freq_of_use',churn_df['freq_of_use'].max())
print('Minimum value in freq_of_use',churn_df['freq_of_use'].min())
print('Maximum value in freq_of_sms',churn_df['freq_of_sms'].max())
print('Minimum value in freq_of_sms',churn_df['freq_of_sms'].min())
print('Maximum value in distinct_call_nums',churn_df['distinct_call_nums'].max())
print('Minimum value in distinct_call_nums',churn_df['distinct_call_nums'].min())

Maximum value in freq_of_use 255.0
Minimum value in freq_of_use -176.0
Maximum value in freq_of_sms 522.0
Minimum value in freq_of_sms -9.0
Maximum value in distinct_call_nums 97.0
Minimum value in distinct_call_nums 0.0


Observation
1. freq_of_use, freq_of_sms has some negative values
2. distinct_call_nums has no negative values

### Imputing null values with mean

These three columns are numerical and are related to the number of calls and SMS sent by consumers, it would be more appropriate to impute null values with the mean value.

In [50]:
# Imputing mean values in place of null values
cols = churn_df.iloc[:,[5,6,7]]
for x in cols.columns:
    mean_column = np.nanmean(churn_df[x].values)
    churn_df[x].replace(np.nan, mean_column, inplace=True)

In [51]:
# Checking for null values in freq_of_use column
print('Number of null values in freq_of_use:',churn_df['freq_of_use'].isnull().sum())
# Checking for null values in freq_of_sms column
print('Number of null values in freq_of_sms:',churn_df['freq_of_sms'].isnull().sum())
# Checking for null values in distinct_call_nums column
print('Number of null values in distinct_call_nums:',churn_df['distinct_call_nums'].isnull().sum())

Number of null values in freq_of_use: 0
Number of null values in freq_of_sms: 0
Number of null values in distinct_call_nums: 0


Observation:

As we can see, now these three columns are free of null values

### Removing -  symbol from the columns

In [52]:
# Eliminating the - symbol since the amount of text messages or phone calls cannot be negative.
for i in cols.columns:
    churn_df[i] = abs(churn_df[i])

In [53]:
#Checking if there are any negative values or extreme values. As these columns cant have negative numbers as values
print('Minimum value in freq_of_use',churn_df['freq_of_use'].min())
print('Minimum value in freq_of_sms',churn_df['freq_of_sms'].min())
print('Minimum value in distinct_call_nums',churn_df['distinct_call_nums'].min())

Minimum value in freq_of_use 0.0
Minimum value in freq_of_sms 0.0
Minimum value in distinct_call_nums 0.0


Observation:
    
We dont have any negative values in the columns 

## Archita

### Correcting incorrect values in age column and then creating a Age group column for clear seggregation of data

In [54]:
churn_df.dtypes

call_failure            int64
complains               int64
subs_len                int64
charge_amount           int64
seconds_of_use          int64
freq_of_use           float64
freq_of_sms           float64
distinct_call_nums    float64
tariff_plan             int64
status                  int64
age                     int64
customer_value        float64
FN                    float64
FP                    float64
Churn                   int64
dtype: object

In [55]:
#displaying distinct values in age column
display(churn_df['age'].value_counts() )

 42     110
 44     102
 34     102
 41     101
 32     100
       ... 
 4        3
 6        2
 233      1
 220      1
-13       1
Name: age, Length: 79, dtype: int64

In [56]:
#adding a new ageGroup column to the data set
churn_df.insert(11, "ageGroup",0)

In [57]:
#calculating the mean of all the ages to replace any incorrect with mean
df_mean = round(churn_df["age"].mean())

In [58]:
df_mean

35

In [59]:
#correcting data in age column by replacing negative or unusual age numbers with mean age
for i in range(len(churn_df)):
    if(churn_df.loc[i,'age'] < 0) | (churn_df.loc[i,'age'] > 100) :
        churn_df.loc[i,'age'] = df_mean 

In [60]:
for i in range(len(churn_df)):
    if  (churn_df.loc[i,'age'] > 0) & (churn_df.loc[i,'age']<=15) :  
        churn_df.loc[i,'ageGroup'] = 1  
    if  (churn_df.loc[i,'age'] > 15) & (churn_df.loc[i,'age']<=30) :  
        churn_df.loc[i,'ageGroup'] = 2 
    if  (churn_df.loc[i,'age'] > 30) & (churn_df.loc[i,'age']<=45) :  
        churn_df.loc[i,'ageGroup'] = 3 
    if  (churn_df.loc[i,'age'] > 45) & (churn_df.loc[i,'age']<=60) :  
        churn_df.loc[i,'ageGroup'] = 4 
    if  (churn_df.loc[i,'age'] > 60) & (churn_df.loc[i,'age']<=80) :  
        churn_df.loc[i,'ageGroup'] = 5
    if  (churn_df.loc[i,'age'] > 80) & (churn_df.loc[i,'age']< 0) :  
        churn_df.loc[i,'ageGroup'] = 2    

In [61]:
  display(churn_df['ageGroup'].value_counts() )

3    1361
2    1130
4     387
5     152
1     111
0       9
Name: ageGroup, dtype: int64

# Aishwarya

In [62]:
# Checking for null values in the dataset
churn_df.isnull().sum()

call_failure          0
complains             0
subs_len              0
charge_amount         0
seconds_of_use        0
freq_of_use           0
freq_of_sms           0
distinct_call_nums    0
tariff_plan           0
status                0
age                   0
ageGroup              0
customer_value        0
FN                    0
FP                    0
Churn                 0
dtype: int64

In [63]:
# Checking if any of the columns is misclassified due to data discrepancies 
churn_df.dtypes

call_failure            int64
complains               int64
subs_len                int64
charge_amount           int64
seconds_of_use          int64
freq_of_use           float64
freq_of_sms           float64
distinct_call_nums    float64
tariff_plan             int64
status                  int64
age                     int64
ageGroup                int64
customer_value        float64
FN                    float64
FP                    float64
Churn                   int64
dtype: object

As we can see, our dataset is free of null values and none of the columns seems to be misclassified as some other type of data we can load the dataset onto a csv file and use it for model building and visualization

In [64]:
churn_df.to_csv('churn_final.csv',index = False)